In [1]:
import os
import pandas as pd
import torch
from FeatureExtraction import ekstraksiFitur

/home/anugerahsurya/tinggikuenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def simpanData(dataObjek, kedalamanObjek, metadata, focalLengthpx, tinggiobj, image_path):
    # Buat DataFrame
    df = pd.DataFrame(dataObjek)
    df['ID'] = image_path
    # Pisahkan center jadi center_x, center_y
    df[['TitikTengah_x', 'TitikTengah_y']] = pd.DataFrame(df['TitikTengah'].tolist(), index=df.index)
    df.drop(columns='TitikTengah', inplace=True)

    # Tambahkan kolom kedalamanObjek
    df['kedalamanObjek'] = kedalamanObjek

    # Ambil nilai float dari tensor focalLengthpx
    fl_value = focalLengthpx.item()

    # Tambahkan kolom fl_px dengan nilai focal length yang sama untuk semua baris
    df['fl_px'] = fl_value

    # Tambahkan kolom metadata dengan fallback ke None jika tidak ditemukan
    df['ImageWidth'] = metadata.get('ImageWidth', metadata.get('ExifImageWidth'))
    df['ImageLength'] = metadata.get('ImageLength', metadata.get('ExifImageHeight'))
    df['FocalLength'] = metadata.get('FocalLength', None)
    df['FocalLengthIn35mmFilm'] = metadata.get('FocalLengthIn35mmFilm', None)
    df['ApertureValue'] = metadata.get('ApertureValue', None)
    df['BrightnessValue'] = metadata.get('BrightnessValue', None)
    df['TinggiAktual'] = tinggiobj
    return df


In [3]:
def run(img_path, tinggiobj, visualize=True):
    dataObjek, kedalamanObjek, metadata, focalLengthpx = ekstraksiFitur(img_path, visualize)

    df = simpanData(dataObjek, kedalamanObjek, metadata, focalLengthpx, tinggiobj, img_path)
    database = '../data/DatabaseTinggi.csv'

    if not os.path.exists(database):
        # File belum ada, simpan baru dengan header
        df.to_csv(database, index=False)
    else:
        # File sudah ada, append tanpa header
        df.to_csv(database, mode='a', index=False, header=False)

In [4]:
# run("../data/Gas/bimo1_180.HEIC",180)

In [5]:
# run("../data/Gas/bimo2_168.HEIC",168)

In [6]:
# run("../data/Gas/bimo3_180.heic",180)

In [7]:
# run("../data/Gas/bimo4_171.jpg",171)

In [8]:
# run("../data/Gas/bimo5_158.JPG",158)

In [9]:
# run("../data/Gas/bimo6_165.heic",165)

In [10]:
# run("../data/Gas/bimo7_163.JPG",163)

In [11]:
# run("../data/Gas/bimo8_176.HEIC",176)

In [12]:
run("../data/Gas/bimo9_176.HEIC",176)


image 1/1 /home/anugerahsurya/Projects/tinggiku/notebooks/../data/Gas/bimo9_176.HEIC: 640x384 1 person, 138.5ms
Speed: 4.5ms preprocess, 138.5ms inference, 572.7ms postprocess per image at shape (1, 3, 640, 384)
Device yang digunakan adalah :  cuda


In [13]:
run("../data/Gas/bimo10_178.jpg",178)


image 1/1 /home/anugerahsurya/Projects/tinggiku/notebooks/../data/Gas/bimo10_178.jpg: 640x480 1 person, 1 tie, 223.7ms
Speed: 10.6ms preprocess, 223.7ms inference, 30.8ms postprocess per image at shape (1, 3, 640, 480)


IndexError: list index out of range

In [ ]:
run("../data/Gas/bimo11_172.jpg",172)

In [ ]:
# run("../data/Gas/bimo12_178.jpg",178) file besar

In [ ]:
# run("../data/Gas/bimo16_170.jpg",170) file besar

In [ ]:
# run("../data/Gas/bimo17_165.jpg",165) file besar

In [ ]:
# run("../data/Gas/bimo18_173.jpg",173) # ga terbaca

In [ ]:
run("../data/Gas/bimo21_180.HEIC",180)

In [ ]:
# run("../data/Gas/bimo22_165.jpg",165) Ga Terbaca

In [ ]:
run("../data/Gas/bimo23_168.jpg",168)

In [ ]:
run("../data/Gas/bimo24_180.heic",180)

In [ ]:
run("../data/Gas/bimo25_167.jpg",167)

In [ ]:
run("../data/Gas/tya_158.jpg",158)

In [ ]:
run("../data/Gas/bimo25_180.HEIC",180)

In [ ]:
run("../data/r_161.jpg",161)

In [ ]:
run("../data/r2_161.HEIC",161)

In [ ]:
run("../data/r3_161.jpg",161)

In [ ]:
run("../data/s1_1765.jpg",176.5)

In [ ]:
run("../data/s2_1765.HEIC",176.5)

In [ ]:
run("../data/s3_1765.jpg",176.5)

/home/anugerahsurya/tinggikuenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Device yang digunakan adalah :  cuda
